In [8]:
import socket
ports = [3000, 8000, 8080]

for port in ports:
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('localhost', port))
        if result == 0:
            print(f'Port {port} is open')
        else:
            print(f'Port {port} is closed')
        sock.close()
    except Exception as e:
        print(f'Error: {e}')


Port 3000 is closed
Port 8000 is open
Port 8080 is closed


In [10]:
import chromadb

try:
    chromadb_client = chromadb.HttpClient(host='localhost', port=8000)
except Exception as e:
    print(f"ChromaDB connection error: {e}")

In [ ]:
import os

import json
from dotenv import load_dotenv
load_dotenv()

CHROMA_HOST = os.getenv("CHROMA_HOST")
CHROMA_PORT = os.getenv("CHROMA_PORT")
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME")
EMBEDDING_MODEL_FORMAT = os.getenv("EMBEDDING_MODEL_FORMAT")
COLLECTION_NAME_DEGREE_PROGRAMS = os.getenv("COLLECTION_NAME_DEGREE_PROGRAMS")
COLLECTION_NAME_COURSES = os.getenv("COLLECTION_NAME_COURSES")


chroma_client = chromadb.HttpClient(
    host='localhost', 
    port=CHROMA_PORT)




collection = chroma_client.create_collection(COLLECTION_NAME_DEGREE_PROGRAMS)

base_data_path = "chroma_loader/"

data_paths = [
    'ug_cat/22_23/',
    'ug_cat/23_24/',
]

for data_path in data_paths:

    files = [f for f in os.listdir(base_data_path + data_path) if f.endswith(".md")]
    
    print(f'Loading data from: {base_data_path + data_path}, found {len(files)} files\n')
    
    # Open the json file that contains embeddings pertaining to each file
    embeddings_json = json.load(open(base_data_path + data_path + f'{EMBEDDING_MODEL_NAME}.json', "r"))

    print(f'Loaded embeddings for {len(embeddings_json.keys())} files\n')    

    content = []
    embeddings = []
    
    for file in files:


        # Get the embedding for the file from the json file
        embeddings.append(embeddings_json[file])

        # Read file content
        with open(base_data_path + data_path + file, "r") as f:
            content.append(f.read())
    
    print(f'Loaded content for {len(content)} files\n')  

    # Add the file ChromaDB collection
    collection.add(
        documents = content,  # content of each doc
        ids = files,  # unique for each doc
        # metadatas=[{"source": doc['source']}],
        embeddings = embeddings
    )

    print(f'Added files to ChromaDB collection\n')



Loading data from: chroma_loader/ug_cat/22_23/, found 45 files

Loaded embeddings for 45 files

Loaded content for 45 files

Added files to ChromaDB collection

Loading data from: chroma_loader/ug_cat/23_24/, found 49 files

Loaded embeddings for 49 files

Loaded content for 49 files

Added files to ChromaDB collection



In [ ]:
import chromadb

def vector_query(
    query_vector_embedding,
    database_type : str,
    db_client,
    db_collection_name,
    limit = 5,
    concat_results = True,) -> str:

    results = []

    # if database_type == 'weaviate':

    #     # Look up the appropriate Weviate database collection - name based on embedding model used
    #     db_collection = db_client.collections.get(db_collection_name)

    #     # Send vector query to database and get response
    #     db_response = db_collection.query.near_vector(
    #         near_vector = query_vector_embedding,
    #         limit = limit,
    #         return_metadata = MetadataQuery(distance=True)
    #     )

    #     for item in db_response.objects:
    #         results.append(item.properties.get('chunk_text'))

    if database_type == 'chroma':
        db_collection = db_client.get_collection(name=db_collection_name)

        # Send vector query to database and get response
        db_response = db_collection.query(
            query_embeddings = query_vector_embedding,
            n_results = limit,
        )
        
        print (db_response['documents'])

        # for item in db_response['documents']:
        #     for subitem in item:
        #         print(subitem)
        #         print("\nSUBITEM BREAK1\n")
        #     results.append(item)

    else:
        print(f"Unsupported database type: {database_type}")
        raise ValueError(f"Unsupported database type: {database_type}")

    if concat_results:

        results_concatenated = "\n<Context_Segment_Break>\n"
        for item in results:

            for subitem in item:
                print(subitem)
                print("\nSUBITEM BREAK2\n")
            results_concatenated += item
            results_concatenated += '\n<Context_Segment_Break>\n'

            

        return results_concatenated
    else:
        return results

In [23]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
CHROMA_HOST = os.getenv("CHROMA_HOST")
CHROMA_PORT = os.getenv("CHROMA_PORT")
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME")
EMBEDDING_MODEL_FORMAT = os.getenv("EMBEDDING_MODEL_FORMAT")
COLLECTION_NAME_DEGREE_PROGRAMS = os.getenv("COLLECTION_NAME_DEGREE_PROGRAMS")
COLLECTION_NAME_COURSES = os.getenv("COLLECTION_NAME_COURSES")

openai.api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI()

content = 'What classes should I take to complete a degree?'

# Get embedding from OpenAI
embedding = openai_client.embeddings.create(
    input=content,
    model=EMBEDDING_MODEL_NAME
)

chroma_client = chromadb.HttpClient(
    host='localhost', 
    port=CHROMA_PORT)

query_result = vector_query(
    query_vector_embedding=embedding.data[0].embedding,
    database_type="chroma",
    db_client = chroma_client,
    db_collection_name= COLLECTION_NAME_DEGREE_PROGRAMS,
    limit=5,
    concat_results=False,
)


# Physics and Electrical Engineering, B.S. Dual Degree

The Department of Physics and Optical Science offers two dual degree opportunities with the Department of Electrical and Computer Engineering. These dual degrees are designed to broaden and enhance the education of students in engineering degree programs. Students can obtain a B.S. Physics and B.S. Electrical Engineering dual degree or a B.S. Physics and B.S. Computer Engineering dual degree.

## Admission Requirements
***
#### Freshmen and Transfers
- _Minimum GPA:_ 2.0 (cumulative and Physics)
- _Pre-Major/Prerequisite Courses:_ Complete the following with grades of C or above and no more than 2 attempts per course: 
  - PHYS 2101
  - PHYS 2102
  - MATH 2241
- _Declaration of Major:_ Engineering majors wishing to declare the B.S. in Physics degree are eligible to do so after meeting the above requirements.

## Degree Requirements
***
To obtain a dual B.S. degree in Electrical Engineering and Physics, undergraduate students must 

In [26]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
CHROMA_HOST = os.getenv("CHROMA_HOST")
CHROMA_PORT = os.getenv("CHROMA_PORT")
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME")
EMBEDDING_MODEL_FORMAT = os.getenv("EMBEDDING_MODEL_FORMAT")
COLLECTION_NAME_DEGREE_PROGRAMS = os.getenv("COLLECTION_NAME_DEGREE_PROGRAMS")
COLLECTION_NAME_COURSES = os.getenv("COLLECTION_NAME_COURSES")

openai.api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI()

content = 'What classes should I take to complete a degree?'

# Get embedding from OpenAI
embedding = openai_client.embeddings.create(
    input=content,
    model=EMBEDDING_MODEL_NAME
).data[0].embedding

chroma_client = chromadb.HttpClient(
    host='localhost', 
    port=CHROMA_PORT)


collection = chroma_client.get_collection(name=COLLECTION_NAME_DEGREE_PROGRAMS)

# Send vector query to database and get response
db_response = collection.query(
    query_embeddings = embedding,
    n_results = 3,
)

print (db_response)


{'ids': [['major_dual_bs_electrial_physics.md', 'major_dual_bs_mech_physics.md', 'major_dual_bs_computer_physics.md']], 'distances': [[0.41578346, 0.41707936, 0.41901135]], 'embeddings': None, 'metadatas': [[None, None, None]], 'documents': [['# Physics and Electrical Engineering, B.S. Dual Degree\n\nThe Department of Physics and Optical Science offers two dual degree opportunities with the Department of Electrical and Computer Engineering. These dual degrees are designed to broaden and enhance the education of students in engineering degree programs. Students can obtain a B.S. Physics and B.S. Electrical Engineering dual degree or a B.S. Physics and B.S. Computer Engineering dual degree.\n\n## Admission Requirements\n***\n#### Freshmen and Transfers\n- _Minimum GPA:_ 2.0 (cumulative and Physics)\n- _Pre-Major/Prerequisite Courses:_ Complete the following with grades of C or above and no more than 2 attempts per course: \n  - PHYS 2101\n  - PHYS 2102\n  - MATH 2241\n- _Declaration of M

In [27]:
print (db_response)

{'ids': [['major_dual_bs_electrial_physics.md', 'major_dual_bs_mech_physics.md', 'major_dual_bs_computer_physics.md']], 'distances': [[0.41578346, 0.41707936, 0.41901135]], 'embeddings': None, 'metadatas': [[None, None, None]], 'documents': [['# Physics and Electrical Engineering, B.S. Dual Degree\n\nThe Department of Physics and Optical Science offers two dual degree opportunities with the Department of Electrical and Computer Engineering. These dual degrees are designed to broaden and enhance the education of students in engineering degree programs. Students can obtain a B.S. Physics and B.S. Electrical Engineering dual degree or a B.S. Physics and B.S. Computer Engineering dual degree.\n\n## Admission Requirements\n***\n#### Freshmen and Transfers\n- _Minimum GPA:_ 2.0 (cumulative and Physics)\n- _Pre-Major/Prerequisite Courses:_ Complete the following with grades of C or above and no more than 2 attempts per course: \n  - PHYS 2101\n  - PHYS 2102\n  - MATH 2241\n- _Declaration of M

In [28]:
print (db_response['documents'])


[['# Physics and Electrical Engineering, B.S. Dual Degree\n\nThe Department of Physics and Optical Science offers two dual degree opportunities with the Department of Electrical and Computer Engineering. These dual degrees are designed to broaden and enhance the education of students in engineering degree programs. Students can obtain a B.S. Physics and B.S. Electrical Engineering dual degree or a B.S. Physics and B.S. Computer Engineering dual degree.\n\n## Admission Requirements\n***\n#### Freshmen and Transfers\n- _Minimum GPA:_ 2.0 (cumulative and Physics)\n- _Pre-Major/Prerequisite Courses:_ Complete the following with grades of C or above and no more than 2 attempts per course: \n  - PHYS 2101\n  - PHYS 2102\n  - MATH 2241\n- _Declaration of Major:_ Engineering majors wishing to declare the B.S. in Physics degree are eligible to do so after meeting the above requirements.\n\n## Degree Requirements\n***\nTo obtain a dual B.S. degree in Electrical Engineering and Physics, undergrad

In [31]:
print(db_response['documents'][0][0])

# Physics and Electrical Engineering, B.S. Dual Degree

The Department of Physics and Optical Science offers two dual degree opportunities with the Department of Electrical and Computer Engineering. These dual degrees are designed to broaden and enhance the education of students in engineering degree programs. Students can obtain a B.S. Physics and B.S. Electrical Engineering dual degree or a B.S. Physics and B.S. Computer Engineering dual degree.

## Admission Requirements
***
#### Freshmen and Transfers
- _Minimum GPA:_ 2.0 (cumulative and Physics)
- _Pre-Major/Prerequisite Courses:_ Complete the following with grades of C or above and no more than 2 attempts per course: 
  - PHYS 2101
  - PHYS 2102
  - MATH 2241
- _Declaration of Major:_ Engineering majors wishing to declare the B.S. in Physics degree are eligible to do so after meeting the above requirements.

## Degree Requirements
***
To obtain a dual B.S. degree in Electrical Engineering and Physics, undergraduate students must 